In [5]:
!pip install pymupdf langchain_openai langchain_elasticsearch elasticsearch

  Using cached pymupdf-1.26.6-cp310-abi3-win_amd64.whl.metadata (3.4 kB)
  Using cached langchain_openai-1.0.3-py3-none-any.whl.metadata (2.6 kB)
  Using cached langchain_elasticsearch-0.4.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached elasticsearch-9.2.0-py3-none-any.whl.metadata (8.9 kB)
  Using cached langchain_core-1.0.5-py3-none-any.whl.metadata (3.6 kB)
  Using cached openai-2.8.0-py3-none-any.whl.metadata (29 kB)
  Using cached tiktoken-0.12.0-cp312-cp312-win_amd64.whl.metadata (6.9 kB)
  Using cached elasticsearch-8.19.2-py3-none-any.whl.metadata (9.1 kB)
INFO: pip is looking at multiple versions of langchain-elasticsearch to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_elasticsearch-0.3.2-py3-none-any.whl.metadata (8.3 kB)
  Using cached langchain_elasticsearch-0.3.1-py3-none-any.whl.metadata (8.3 kB)
  Using cached langchain_elasticsearch-0.3.0-py3-none-any.whl.metadata (8.3 kB)
INFO: pip is still lookin

In [8]:
!pip install langchain-text-splitters

  Using cached langchain_core-1.0.5-py3-none-any.whl.metadata (3.6 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.79
    Uninstalling langchain-core-0.3.79:
      Successfully uninstalled langchain-core-0.3.79


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-elasticsearch 0.4.0 requires langchain-core<0.4.0,>=0.3.0, but you have langchain-core 1.0.5 which is incompatible.
langchain-openai 0.3.35 requires langchain-core<1.0.0,>=0.3.78, but you have langchain-core 1.0.5 which is incompatible.


In [58]:
import fitz  # PyMuPDF
import pprint # Para imprimir bonito
from pathlib import Path# Tu Endpoint (ya incluye https:// y el puerto 443 está implícito)

In [59]:
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_elasticsearch import ElasticsearchStore

In [60]:
PDF_PATH = Path("data/cfa_books/V6 - Fixed Income (CFA-L1-2025).pdf")
NEW_INDEX_NAME = "cfa_documents_structured" # ¡Usaremos un índice nuevo!

# Configuración del "chunker"
CHUNK_SIZE = 1200
CHUNK_OVERLAP = 250

In [61]:
print("✅ Librerías y constantes cargadas.")
print(f"Libro a procesar: {PDF_PATH.name}")
print(f"Índice de destino: {NEW_INDEX_NAME}")

✅ Librerías y constantes cargadas.
Libro a procesar: V6 - Fixed Income (CFA-L1-2025).pdf
Índice de destino: cfa_documents_structured


In [62]:
import fitz
import re

print(f"Abriendo: {PDF_PATH.name} para extraer el índice...")
doc = fitz.open(PDF_PATH)
toc = doc.get_toc() # Intenta la extracción automática


Abriendo: V6 - Fixed Income (CFA-L1-2025).pdf para extraer el índice...


In [63]:
toc

[[1, 'How to Use the CFA Program Curriculum', 9],
 [2, 'CFA Institute Learning Ecosystem (LES)', 9],
 [3, 'Designing Your Personal Study Program', 9],
 [3, 'Errata', 10],
 [3, 'Other Feedback', 10],
 [1, 'Fixed Income', 11],
 [2, 'Learning Module 1\tFixed-Income Instrument Features', 13],
 [3, 'Introduction', 13],
 [3, 'Features of Fixed-Income Securities', 16],
 [4, 'Issuer', 18],
 [4, 'Maturity', 18],
 [4, 'Principal (Par or Face Value)', 19],
 [4, 'Coupon Rate and Frequency', 19],
 [4, 'Seniority', 20],
 [4, 'Contingency Provisions', 20],
 [4, 'Yield Measures', 21],
 [4, 'Yield Curves', 21],
 [3, 'Bond Indentures and Covenants', 24],
 [4, 'Bond Indentures', 24],
 [4, 'Sources of Repayment', 24],
 [4, 'Bond Covenants', 26],
 [3, 'Practice Problems', 30],
 [3, 'Solutions', 32],
 [2, 'Learning Module 2\tFixed-Income Cash Flows and Types', 33],
 [3, 'Introduction', 33],
 [3, 'Fixed-Income Cash Flow Structures', 36],
 [4, 'Amortizing Debt', 37],
 [4, 'Variable Interest Debt', 42],
 [4, '

In [56]:
# Célula 2: Extraer el Índice (ToC) y Crear el "Mapa" Jerárquico

if not toc:
    print("❌ PDF no tiene ToC automático. Usando Plan B (RegEx)...")
    toc_text = ""
    # Escanear las páginas del índice (ajusta el rango si es necesario)
    for page_num in range(2, 10): 
        toc_text += doc.load_page(page_num).get_text()
    
    # Patrón RegEx para encontrar "TOPIC..." o "Reading..."
    pattern = re.compile(r'^((?:TOPIC|Reading) \d+:[^.\n]+)\.{5,}\s*(\d+)$', re.MULTILINE)
    
    toc = []
    for match in pattern.finditer(toc_text):
        title = match.group(1).strip()
        page = int(match.group(2))
        # Asignamos un "nivel" basado en el texto
        level = 1 if title.startswith("TOPIC") else 2
        toc.append([level, title, page])

if not toc:
    print("❌ ERROR: No se pudo extraer el índice. No se pueden crear metadatos.")
else:
    # --- Lógica Jerárquica ---
    section_map = []
    current_main_topic = "Unknown" # Variable de estado para el Tema
    
    for level, title, page in toc:
        if title.startswith("TOPIC"):
            # 1. Si es un TEMA, actualizamos nuestra variable de estado
            current_main_topic = title
        elif title.startswith("Reading"):
            # 2. Si es una LECTURA, la añadimos al mapa *con* su Tema padre
            section_map.append({
                "main_topic": current_main_topic,
                "reading": title,
                "start_page": page
            })
    
    print("\n--- 🗺️ Mapa de Secciones Creado (Nivel Lectura) ---")
    pprint.pprint(section_map)
    print("--------------------------------------------------")

Abriendo: V6 - Fixed Income (CFA-L1-2025).pdf para extraer el índice...

--- 🗺️ Mapa de Secciones Creado (Nivel Lectura) ---
[]
--------------------------------------------------


In [57]:
toc

[[1, 'How to Use the CFA Program Curriculum', 9],
 [2, 'CFA Institute Learning Ecosystem (LES)', 9],
 [3, 'Designing Your Personal Study Program', 9],
 [3, 'Errata', 10],
 [3, 'Other Feedback', 10],
 [1, 'Fixed Income', 11],
 [2, 'Learning Module 1\tFixed-Income Instrument Features', 13],
 [3, 'Introduction', 13],
 [3, 'Features of Fixed-Income Securities', 16],
 [4, 'Issuer', 18],
 [4, 'Maturity', 18],
 [4, 'Principal (Par or Face Value)', 19],
 [4, 'Coupon Rate and Frequency', 19],
 [4, 'Seniority', 20],
 [4, 'Contingency Provisions', 20],
 [4, 'Yield Measures', 21],
 [4, 'Yield Curves', 21],
 [3, 'Bond Indentures and Covenants', 24],
 [4, 'Bond Indentures', 24],
 [4, 'Sources of Repayment', 24],
 [4, 'Bond Covenants', 26],
 [3, 'Practice Problems', 30],
 [3, 'Solutions', 32],
 [2, 'Learning Module 2\tFixed-Income Cash Flows and Types', 33],
 [3, 'Introduction', 33],
 [3, 'Fixed-Income Cash Flow Structures', 36],
 [4, 'Amortizing Debt', 37],
 [4, 'Variable Interest Debt', 42],
 [4, '

In [27]:
# Célula 3: Chunking Estructural (Stateful Chunking)

if not toc:
    print("❌ No se puede continuar sin el 'Mapa de Secciones' del Paso 2.")
else:
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=CHUNK_SIZE,
        chunk_overlap=CHUNK_OVERLAP,
        length_function=len,
        separators=["\n\n## ", "\n\n### ", "\n\n", "\n", ". ", " ", ""]
    )

    chunks = []
    current_topic = "Introducción / Prefacio" # Tema por defecto
    map_index = 0

    print(f"Procesando {doc.page_count} páginas...")

    # Iterar por cada página del PDF
    for page_num in range(doc.page_count):
        page_number = page_num + 1 # Las páginas son 1-indexed

        # 1. Actualizar el "estado" (el tema actual)
        if map_index < len(section_map) and page_number >= section_map[map_index]["start_page"]:
            current_topic = section_map[map_index]["topic"]
            print(f"  -> Entrando al Tema: '{current_topic}' (Página {page_number})")
            map_index += 1

        # 2. Extraer el texto de la página
        page_text = doc.load_page(page_num).get_text("text")

        # 3. Dividir el texto de ESTA PÁGINA en chunks
        page_chunks = text_splitter.split_text(page_text)

        # 4. Etiquetar cada chunk con la metadata "inteligente"
        for chunk_text in page_chunks:
            metadata = {
                "source_file": PDF_PATH.name,
                "page_number": page_number,
                "topic": current_topic,
                "cfa_level": "I" # Puedes extraer esto del nombre del archivo también
            }
            
            # Crear el Documento de LangChain
            new_doc = Document(page_content=chunk_text, metadata=metadata)
            chunks.append(new_doc)

    print("\n--- ✅ Chunking Estructural Completado ---")
    print(f"Total de Chunks Creados: {len(chunks)}")
    
    print("\n--- Ejemplo de Chunk Inteligente (Metadatos) ---")
    pprint.pprint(chunks[100].metadata)
    print("----------------------------------------------")

Procesando 360 páginas...
  -> Entrando al Tema: 'How to Use the CFA Program Curriculum' (Página 7)
  -> Entrando al Tema: 'Quantitative Methods' (Página 9)

--- ✅ Chunking Estructural Completado ---
Total de Chunks Creados: 911

--- Ejemplo de Chunk Inteligente (Metadatos) ---
{'cfa_level': 'I',
 'page_number': 40,
 'source_file': 'V1 - Quantitive Methods (CFA-L1-2025).pdf',
 'topic': 'Quantitative Methods'}
----------------------------------------------


In [29]:
# Célula 4: Crear el Mapeo (Mapping) del Índice
es_client = get_elasticsearch_client()

if es_client:
    print(f"Conectado a Elasticsearch. Preparando índice: {NEW_INDEX_NAME}")

    index_mapping = {
        "properties": {
            "vector": {
                "type": "dense_vector",
                "dims": 1536, # ej. 1536
                "index": True,
                "similarity": "cosine"
            },
            "text": {"type": "text"},
            
            # --- NUESTROS METADATOS FILTRABLES ---
            "topic": {"type": "keyword"},
            "cfa_level": {"type": "keyword"},
            "page_number": {"type": "integer"},
            "source_file": {"type": "keyword"}
            # ------------------------------------
        }
    }

    # Eliminar el índice si ya existe (para empezar de cero)
    try:
        if es_client.indices.exists(index=NEW_INDEX_NAME):
            print(f"🗑️ Eliminando índice antiguo: {NEW_INDEX_NAME}")
            es_client.indices.delete(index=NEW_INDEX_NAME)
        
        print(f"🔨 Creando nuevo índice: {NEW_INDEX_NAME}")
        es_client.indices.create(index=NEW_INDEX_NAME, mappings=index_mapping)
        print("✅ Índice creado exitosamente.")
        
    except Exception as e:
        print(f"❌ Error al configurar el índice: {e}")
else:
    print("❌ Error: No se pudo conectar a Elasticsearch.")

C:\Users\fguerrero\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cfaagent-2adb7b.es.us-central1.gcp.cloud.es.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\fguerrero\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cfaagent-2adb7b.es.us-central1.gcp.cloud.es.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


✅ Conectado a Elasticsearch en https://cfaagent-2adb7b.es.us-central1.gcp.cloud.es.io
   Cluster: eb01aeae4cff48b69fe8ac82449abf60
   Versión: 9.2.1
Conectado a Elasticsearch. Preparando índice: cfa_documents_structured


C:\Users\fguerrero\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cfaagent-2adb7b.es.us-central1.gcp.cloud.es.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\fguerrero\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cfaagent-2adb7b.es.us-central1.gcp.cloud.es.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


🔨 Creando nuevo índice: cfa_documents_structured
✅ Índice creado exitosamente.


In [20]:
def get_elasticsearch_client():
    """Crea y retorna un cliente de Elasticsearch configurado."""
    from elasticsearch import Elasticsearch
    
    try:
        es_client = Elasticsearch(
            [ES_URL],
            basic_auth=(ES_USERNAME, ES_PASSWORD),
            verify_certs=False,  # ⚠️ En producción, usa certificados válidos
            request_timeout=30,
            max_retries=3,
            retry_on_timeout=True
        )
        
        if es_client.ping():
            print(f"✅ Conectado a Elasticsearch en {ES_URL}")
            info = es_client.info()
            print(f"   Cluster: {info['cluster_name']}")
            print(f"   Versión: {info['version']['number']}")
            return es_client
        else:
            print(f"❌ No se pudo conectar a Elasticsearch en {ES_URL}")
            return None
    
    except Exception as e:
        print(f"❌ Error conectando a Elasticsearch: {e}")
        return None


In [ ]:
c

In [22]:
# Tu Endpoint (ya incluye https:// y el puerto 443 está implícito)
ES_URL = "https://cfaagent-2adb7b.es.us-central1.gcp.cloud.es.io"

# Tu usuario (casi siempre es 'elastic' en estos deployments)
ES_USERNAME = "elastic" 

# ¡Pega tu contraseña aquí!
ES_PASSWORD = "HbTCaDhJv4R4UjA0jSHGZnCn" #

In [30]:
# Célula 6: Verificar la Indexación
import json

if es_client and chunks:
    print(f"Verificando el contenido del índice {NEW_INDEX_NAME}...")
    
    # Refrescar el índice para asegurar que los documentos sean buscables
    es_client.indices.refresh(index=NEW_INDEX_NAME)
    
    # Contar documentos
    count_response = es_client.count(index=NEW_INDEX_NAME)
    doc_count = count_response.get('count', 0)
    print(f"Total de documentos en el índice: {doc_count} (vs {len(chunks)} chunks esperados)")
    
    # Obtener un documento de muestra
    search_response = es_client.search(
        index=NEW_INDEX_NAME,
        size=1,
        from_=100 # Tomar una muestra del medio
    )
    
    hits = search_response.get('hits', {}).get('hits', [])
    if hits:
        print("\n--- 📝 Muestra de Documento Indexado ---")
        # El documento está en el campo '_source'
        sample_doc = hits[0]['_source']
        
        # Imprimir solo el texto y la metadata
        print(f"Texto: {sample_doc['text'][:150]}...")
        
        # ¡LA PRUEBA CLAVE!
        print("\n--- 🔑 METADATA ---")
        print(f"  Fuente: {sample_doc.get('source_file')}")
        print(f"  Página: {sample_doc.get('page_number')}")
        print(f"  Tema:   {sample_doc.get('topic')}")
        print(f"  Nivel:  {sample_doc.get('cfa_level')}")
        print("--------------------")
    else:
        print("❌ No se encontraron documentos en el índice.")

Verificando el contenido del índice cfa_documents_structured...


C:\Users\fguerrero\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cfaagent-2adb7b.es.us-central1.gcp.cloud.es.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\fguerrero\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cfaagent-2adb7b.es.us-central1.gcp.cloud.es.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Total de documentos en el índice: 0 (vs 911 chunks esperados)
❌ No se encontraron documentos en el índice.


C:\Users\fguerrero\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cfaagent-2adb7b.es.us-central1.gcp.cloud.es.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [49]:
# Célula 5: Indexar los Chunks
if es_client and chunks:
    print("🧠 Inicializando embeddings de OpenAI...")
    embeddings = OpenAIEmbeddings(
        model=EMBEDDING_MODEL,
        openai_api_key=OPENAI_API_KEY,
        chunk_size=1000 # Lotes de 1000 a la API de OpenAI
    )

    print(f"📤 Indexando {len(chunks)} chunks en Elasticsearch...")
    print("(Esto puede tardar varios minutos...)")
    
    try:
        # Usar la configuración de Cloud ID / API Key (más robusto)
        es_config = get_es_config()

        vector_store = ElasticsearchStore.from_documents(
            documents=chunks,
            embedding=embeddings,
            index_name=NEW_INDEX_NAME,
            
            # --- Conexión Recomendada ---
            es_cloud_id=es_config.get("es_cloud_id"),
            es_api_key=es_config.get("es_api_key"),
            
            # --- (Alternativa) Conexión URL/User/Pass ---
            # es_url=es_config.get("es_url"),
            # es_user=es_config.get("es_user"),
            # es_password=es_config.get("es_password"),

            # --- ¡La solución a tu error de Timeout! ---
            bulk_kwargs={
                "request_timeout": 120  # Esperar 2 minutos
            }
        )
        
        print("\n--- ✅ ¡INDEXACIÓN COMPLETADA! ---")

    except Exception as e:
        print(f"\n❌ ERROR DURANTE LA INDEXACIÓN: {e}")
        import traceback
        traceback.print_exc()

🧠 Inicializando embeddings de OpenAI...
📤 Indexando 911 chunks en Elasticsearch...
(Esto puede tardar varios minutos...)

❌ ERROR DURANTE LA INDEXACIÓN: Please provide either elasticsearch_url or cloud_id.


Traceback (most recent call last):
  File "C:\Users\fguerrero\AppData\Local\Temp\ipykernel_25384\3119149125.py", line 17, in <module>
    vector_store = ElasticsearchStore.from_documents(
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\fguerrero\AppData\Local\anaconda3\Lib\site-packages\langchain_elasticsearch\_sync\vectorstores.py", line 786, in from_documents
    elasticsearchStore = cls(embedding=embedding, **kwargs)
                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\fguerrero\AppData\Local\anaconda3\Lib\site-packages\langchain_elasticsearch\_sync\vectorstores.py", line 336, in __init__
    es_connection = create_elasticsearch_client(
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\fguerrero\AppData\Local\anaconda3\Lib\site-packages\langchain_elasticsearch\client.py", line 26, in create_elasticsearch_client
    raise ValueError("Please provide either elasticsearch_url or cloud_id.")
ValueError: Please provide either 

In [35]:
EMBEDDING_MODEL = "text-embedding-3-large"


In [40]:

def get_es_config() -> dict:
    """Retorna configuración para ElasticsearchStore de LangChain."""
    return {
        "es_url": ES_URL,
        "es_user": ES_USERNAME,
        "es_password": ES_PASSWORD,
        "index_name": ES_INDEX_NAME
    }


In [42]:
ES_INDEX_NAME = "cfa_documents_structured" # ¡Usaremos un índice nuevo!

In [48]:
# Tu Endpoint (ya incluye https:// y el puerto 443 está implícito)
ES_URL = "https://cfaagent-2adb7b.es.us-central1.gcp.cloud.es.io"
